# Bring in Modules

In [1]:
import pandas as pd

# Bring in the Data

In [2]:
cord_19_metadata = pd.read_csv("/Volumes/LaCie/Dispersed Volunteer Research Network/Project 21/Data/2021-03-29/metadata.csv")
fda_biologics = pd.read_csv("/Volumes/LaCie/Dispersed Volunteer Research Network/Project 21/Data/FDA Products List (Last Updated ~September 2020) - Sheet1.csv")

/Users/cpollack/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Creating Processed List of FDA Biologics Data

In [29]:
fda_biologics["processed"] = fda_biologics.PRODUCT.str.replace("_", " ")
fda_biologics.processed = fda_biologics.processed.str.lower()
fda_biologics_list = [y for x in fda_biologics.processed.tolist() for y in x.split('/')]

# Filtering to only include COVID-19 related terms

## Terms  pulled from NIH MeSH: https://www.ncbi.nlm.nih.gov/mesh/2052179

In [37]:
covid_19_terms = ["COVID 19", "COVID-19", "2019-nCoV", "2019 nCoV", "Coronavirus Disease-19", "Coronavirus Disease 19", " 2019 Novel Coronavirus",
                  "COVID19", "Coronavirus Disease 2019", "SARS Coronavirus 2 Infection", "SARS-CoV-2", "SARS CoV 2"]

## Processing abstract data

In [63]:
cord_19_metadata["processed_abstract"] = cord_19_metadata.abstract.str.lower()
print(cord_19_metadata.shape) #497,906 rows before
cord_19_metadata = cord_19_metadata[cord_19_metadata.processed_abstract.notna()]
print(cord_19_metadata.shape) #363,064 rows after removing NaN abstracts

/Users/cpollack/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(363064, 20)
(363064, 20)


In [82]:
cord_19_covid = cord_19_metadata[cord_19_metadata["processed_abstract"].str.contains('|'.join([x.lower() for x in covid_19_terms]))]
print(cord_19_covid.shape) #198,494 when only keeping those that directly mention COVID-19

(198494, 20)


# Filtering to only include abstracts related to biologics of interest

In [83]:
cord_19_covid_fda = cord_19_metadata[cord_19_metadata["processed_abstract"].str.contains('|'.join(fda_biologics_list))]
print(cord_19_covid_fda.shape) #198,494 when only keeping those that directly mention COVID-19

KeyboardInterrupt: 

# Next steps:
- Finish running above code 
- EDA
- Take a random sample of 1,000 abstracts to test accuracy/precision of COVID-19 search terms